In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
!pip install pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=cf8d7579d2d0be78e4d9db9cfcff3c9085f96de19a73f1d2d3670245b57b304a
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


In [8]:
from models.ddfm import *

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [10]:
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [11]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/sfr_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/sfr_test.csv')
df_train_idx=df_train.set_index("index")
df_test_idx=df_test.set_index("index")

In [13]:
df_test_idx.sort_index(inplace=True)
mean_test = df_test_idx.mean().values
sigma_test = df_test_idx.std().values
data_test = (df_test_idx - mean_test ) / sigma_test

# Para 7 factores:

In [36]:
r_true=7
structure_encoders = [(r_true * 6, r_true * 4, r_true * 2, r_true),
                      (r_true * 12, r_true * 6,r_true * 4, r_true)]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Fit the model
            ddfm.fit()

            # Store the model, its configuration, and final loss
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss))

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.5618317276176635 - delta: 0.00025093903509039857 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5665182590611652 - delta: 0.00038915767374187116 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5605203296806641 - delta: 0.00037746011803267015 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5692886499619476 - delta: 0.0003273819488660052 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info - Note: Sorting data.
19/19 [================

In [37]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    print(f"Minimum Loss: {min_loss} with encoder {min_loss_config[0]}, lag {min_loss_config[1]}, and use_bias {min_loss_config[2]}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.5605203296806641 with encoder (84, 42, 28, 7), lag 1, and use_bias True


In [38]:
ddfm7 = DDFM(df_train_idx, structure_encoder=(84, 42, 28, 7), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm7.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5602081128154917 - delta: 0.00035190491042899163 < 0.0005


In [39]:
last_state, last_state_var_cov_matrix = ddfm7.get_last_state()

In [40]:
steps_ahead =12  # for example, predict 5 steps ahead
predictions7 = ddfm7.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_7=ddfm7.predict_observations(last_state,last_state_var_cov_matrix , steps_ahead)
df_test_first_12_rows = data_test.iloc[:12, :]
test_array12 = df_test_first_12_rows.to_numpy()
mse_row12=np.mean((test_array12 - y_t_k_7)**2, axis=1)
mse_12=np.mean(mse_row12)
mae_12 = np.mean(np.abs(test_array12 - y_t_k_7))
rmsfe_12 = np.sqrt(np.mean((test_array12 - y_t_k_7) ** 2))
print("Mean Squared Error:", mse_12)
print("Mean Absolute Error:", mae_12)
print("Root Mean Squared Forecast Error:", rmsfe_12)

Mean Squared Error: 3.232841019962182
Mean Absolute Error: 1.262717970876207
Root Mean Squared Forecast Error: 1.79801029473198


In [41]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions7 = ddfm7.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_7=ddfm7.predict_observations(last_state, last_state_var_cov_matrix, steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = data_test.to_numpy()
mse_row=np.mean((test_array - y_t_k_7)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_7))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_7) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9742071554240886
Mean Absolute Error: 0.6889207397034728
Root Mean Squared Forecast Error: 0.9870193287996383


# Para 6 factores:

In [21]:
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r)]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Fit the model
            ddfm.fit()

            # Store the model, its configuration, and final loss
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss))

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5995689419402952 - delta: 0.0003461674847650658 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.5924710833917752 - delta: 0.0004006893896459243 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.5980999299260512 - delta: 0.0003835323473256483 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.590151673341049 - delta: 0.00045944281729073906 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.603700714247204 - delta: 3.0210114676002476e-05 < 0.0005
@Info - Note: Sortin

In [22]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    print(f"Minimum Loss: {min_loss} with encoder {min_loss_config[0]}, lag {min_loss_config[1]}, and use_bias {min_loss_config[2]}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.590151673341049 with encoder (72, 36, 24, 6), lag 1, and use_bias True


In [23]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(72, 36, 6), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5897180859958142 - delta: 0.00036826724971821743 < 0.0005


In [24]:
last_state, last_state_var_cov_matrix = ddfm6.get_last_state()

In [25]:
last_state.shape

(124,)

In [27]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions6 = ddfm6.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_6=ddfm6.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = data_test.to_numpy()
mse_row=np.mean((test_array - y_t_k_6)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_6))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_6) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.977012204749628
Mean Absolute Error: 0.6897566487588299
Root Mean Squared Forecast Error: 0.9884392772192069


In [ ]:
#n = ddfm.state_space.F.shape[0]
#x_hat_start = np.zeros((1, n))
#sigma_x_hat_start = np.eye(ddfm.state_space.F.shape[0])
#x_hat_start.shape

In [ ]:
#dir(ddfm)

### factores filtrados

In [28]:
ff6=ddfm6.factors_filtered
ff6.shape

(577, 6)

In [29]:
f_hat_ddfm_idx6 = pd.DataFrame(ff6, columns=["f1", "f2", "f3", "f4", "f5", "f6"])

In [30]:
f_hat_ddfm_idx6.head()

,f1,f2,f3,f4,f5,f6
0,-0.528321,0.001862,0.614600,0.651493,0.206933,-0.940086
1,0.248153,0.200549,-0.033575,-0.038581,-0.618938,0.063320
2,-0.266969,0.270688,-0.138920,0.006120,-0.694777,0.499269
3,0.162667,-0.085176,0.053944,-0.066591,-0.455956,0.378463
4,-0.187374,0.403998,-0.040056,0.002382,0.025598,0.206293


In [31]:
f_hat_ddfm_idx6.index = df_train_idx.index
f_hat_ddfm6=f_hat_ddfm_idx6.reset_index()

In [33]:
f_hat_ddfm6.head()

,index,f1,f2,f3,f4,f5,f6
0,1960-01-01,-0.528321,0.001862,0.614600,0.651493,0.206933,-0.940086
1,1960-02-01,0.248153,0.200549,-0.033575,-0.038581,-0.618938,0.063320
2,1960-03-01,-0.266969,0.270688,-0.138920,0.006120,-0.694777,0.499269
3,1960-04-01,0.162667,-0.085176,0.053944,-0.066591,-0.455956,0.378463
4,1960-05-01,-0.187374,0.403998,-0.040056,0.002382,0.025598,0.206293


In [35]:
f_hat_ddfm_idx6.to_csv('/content/drive/MyDrive/D2FM/f_hat_ddfm_fact6.csv', index=False)